# Problem 5.1

In [ ]:
'''
You can start using the same code as in HW3. Some things to consider:

- Now we are using the skorch library which allows us to do parameter tunning in the same way as for
other classifiers in sklearn. 
- The code to add L1 regularization is already implemented.


'''

In [1]:
'''
Import packages 
'''

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
import torch.nn.functional as F

# Device configuration - If you have CUDA configured, you must use it. Try training with CPU and observe what happens
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


'''
Step - Import Mnist training and testing data sets as we did for HW3. But do not use Dataloader, 
skorch will do that step for us.

Step - Normalize the data so the pixel values are between 0 and 1. 
     - Make sure the shape of the data tensor is not (n_observations, 28, 28), but (n_observations, 784)
     - Also make sure the data is type float and the targets type integer
     - Name the data X_train and X_test and the labels y_train and y_test
     
'''


'\nStep - Import Mnist training and testing data sets as we did for HW3. But do not use Dataloader, \nskorch will do that step for us.\n\nStep - Normalize the data so the pixel values are between 0 and 1. \n     - Make sure the shape of the data tensor is not (n_observations, 28, 28), but (n_observations, 784)\n     - Also make sure the data is type float and the targets type integer\n     - Name the data X_train and X_test and the labels y_train and y_test\n     \n'

In [2]:
import numpy as np

In [3]:
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
X_train = torch.cat(tuple([train_dataset[i][0].reshape(-1,28*28) for i in range(len(train_dataset))]))
X_train.shape

torch.Size([60000, 784])

In [5]:
X_test = torch.cat(tuple([test_dataset[i][0].reshape(-1,28*28) for i in range(len(test_dataset))]))
X_test.shape

torch.Size([10000, 784])

In [6]:
y_train = torch.tensor([train_dataset[i][1] for i in range(len(train_dataset))])
y_train.shape

torch.Size([60000])

In [7]:
y_test = torch.tensor([test_dataset[i][1] for i in range(len(test_dataset))])
y_test.shape

torch.Size([10000])

In [8]:
ytrain = y_train.type(torch.LongTensor)
y_test = y_test.type(torch.LongTensor)

In [9]:
#Adding L1 regularization

from skorch import NeuralNet

class MyModule(nn.Module):
    def __init__(self, input_size, layer1size, layer2size, num_classes, dropout=0.25):
        #self.__hLL = hiddenLayerList
        super(MyModule, self).__init__()
        
        #First, we will take as input
        self.first = nn.Linear(input_size, layer1size)
        
        #Here's the ReLU activation.
        self.relu = nn.ReLU(inplace=False)
        
        self.second = nn.Linear(layer1size, layer2size)
            
        #And now we go from the last hidden layer to the output layer.
        #self.last = nn.Linear(hiddenLayerList[-1], num_classes)
        self.last = nn.Linear(layer2size, num_classes)
        
        #Here is the last activation softmax
        self.Last = nn.LogSoftmax()
        
        #Lastly, we'll define dropout.
        self.dropout = nn.Dropout(dropout)
        
    #Now we'll define the forward method.
    def forward(self, x):
        out = self.first(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.second(out)
        out = self.relu(out)
        self.dropout(out)
        out = self.last(out)
        out = self.Last(out)
        return out
    
    '''
    
    Step - Fill in the Neural Network here. Use the same arquitecture as in HW3, but now add a new hidden layer.
    and dropout
    Remember to use ReLU activations in the hidden layers. 
    
    
    
    '''
    
    pass



In [ ]:
# We'll try a few loss functions:
# criterion = nn.CrossEntropyLoss()

In [10]:
class RegularizedNet(NeuralNet):
    
    ''''''
    
    def __init__(self, *args, lambda1=0.01, **kwargs):
        super().__init__(*args, **kwargs)
        self.lambda1 = lambda1
    
    ''' *** - What is the following method doing? Explain in detail in the main pdf ***'''
    
    def get_loss(self, y_pred, y_true, X=None, training=False):
        loss = super().get_loss(y_pred, y_true, X=X, training=training)
        loss += self.lambda1 * sum([w.abs().sum() for w in self.module_.parameters()])
        return loss

    

In [ ]:
# torch.optim.SGD(model.parameters(), lr = learning_rate)

In [16]:
'''
Here we define the RegularizedNet. Make sure you use nn.NLLLoss. Thus, you have to use a correct last activation
in the forward method of your network

We can especify different parameters as learning rate (lr), our optimizar (start with standard SGD, in 5.3 we will
try another ones), batch size etc.
To define the arquitecture parameters for MyModule write them as module__<name of your parameter> = ....

Since we have to train it first with L2 regularization lambda1 should be equal to 0
'''
new_net = RegularizedNet(module=MyModule, criterion=torch.nn.NLLLoss, 
                        optimizer=torch.optim.SGD, lr = 0.15, lambda1 = 0,  module__dropout = 0.1,
                        optimizer__weight_decay = 0, module__input_size = 784, module__layer1size = 400,
                        module__layer2size = 150, module__num_classes = 10)
    

In [17]:
'''Step - train the network'''

new_net.fit(X_train, y_train, epochs=20)

'''
Look how your loss is going down as well as the validation accuracy is increasing 
'''

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.6510        0.3073  3.1279
      2        0.2432        0.2330  3.1647
      3        0.1712        0.1747  3.4767
      4        0.1312        0.1425  3.2860
      5        0.1052        0.1125  3.4352
      6        0.0866        0.1015  3.4442
      7        0.0728        0.0939  3.1891
      8        0.0616        0.0853  3.4607
      9        0.0540        0.0875  3.2616
     10        0.0469        0.0801  3.2887
     11        0.0402        0.0773  3.2735
     12        0.0343        0.0786  3.3048
     13        0.0302        0.0744  3.3986
     14        0.0268        0.0752  3.3385
     15        0.0245        0.0719  3.4318
     16        0.0217        0.0697  3.6788
     17        0.0184        0.0730  3.4992
     18        0.0152        0.0718  3.6712
     19        0.0150        0.0730  3.3920
     20        0.0142        0.0706  3.3796


'\nLook how your loss is going down as well as the validation accuracy is increasing \n'

In [18]:
y_pred_probs = new_net.predict(X_test)

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


In [19]:
'''Step - Predict for the test set and print the final accuracy score, your validation accuracy obtained in the previous
cell should be similar to the accuracy in the test set
'''
# y_pred = new_net.predict(X_test)
y_pred = torch.tensor([np.argmax(y_pred_probs[i]) for i in range(y_pred_probs.shape[0])])
y_pred = y_pred.type(torch.LongTensor)

In [20]:
accuracy_score(y_test, y_pred)

0.9801

In [21]:
'''The idea is that you should get more than 98% of accuracy, so try different parameters as requested in the main pdf
The fit method is already showing you a validation error which can be used to compare between different parameters.

for the final submission leave the best parameters in your RegularizedNet(...)
'''
'''
Instead of doing it manually skorch allows us to use GridSearchCV from sklearn 
'''
from sklearn.model_selection import GridSearchCV

'''
Step - define a grid with some parameters that you consider may give you good results and 
the code will do the rest for you

* Especially take into account the parameters we are asking for to tune:  
Learning rate, regularization parameter, and the number of nodes

'''




'\nStep - define a grid with some parameters that you consider may give you good results and \nthe code will do the rest for you\n\n* Especially take into account the parameters we are asking for to tune:  \nLearning rate, regularization parameter, and the number of nodes\n\n'

In [50]:
#module=MyModule, criterion=torch.nn.NLLLoss, 
 #                       optimizer=torch.optim.SGD, lr = 0.13, lambda1 = 0,  module__dropout = 0.25,
  #                      optimizer__weight_decay = 0, module__input_size = 784, module__layer1size = 400,
   #                     module__layer2size = 150, module__num_classes = 10


grid = {
    'lr': [0.01, 0.1, 0.001],
    'module__layer1size': [400,200],
    'module__layer2size': [150,70],
    'optimizer__weight_decay': [0.001, 0.1]
    
}





In [ ]:
#Due to issues with datatypes, we will create our own scoring method.

In [46]:
def my_scoring(Estimator, X_Test, Y_Test):
    Y_pred_probs = Estimator.predict(X_Test)
    Y_pred = torch.tensor([np.argmax(Y_pred_probs[i]) for i in range(Y_pred_probs.shape[0])])
    Y_pred = Y_pred.type(torch.LongTensor)
    return accuracy_score(Y_Test, Y_pred)


In [51]:
'''
Important that you keep refit = True
'''
#gs = GridSearchCV(new_net, grid, refit=True, cv=5, scoring='accuracy')
gs = GridSearchCV(new_net, grid, refit=True, cv=5, scoring=my_scoring)

In [89]:
'''
Finally fit
'''
gs.fit(X_train, y_train, epochs=20)



<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        2.2240        2.1012  2.2715
      2        1.7688        1.3284  2.2897
      3        0.9892        0.7730  2.2907
      4        0.6591        0.5925  2.3082
      5        0.5367        0.5080  2.8008
      6        0.4721        0.4584  2.8426
      7        0.4316        0.4259  2.7422
      8        0.4053        0.4026  2.7155
      9        0.3840        0.3848  2.6445
     10        0.3669        0.3704  2.5940
     11        0.3515        0.3591  2.7211
     12        0.3410        0.3490  2.7527
     13        0.3302        0.3393  2.6141
     14        0.3210        0.3311  2.5949
     15        0.3108        0.3229  2.6767
     16        0.3052        0.3157  2.5555
     17        0.2956        0.3094  2.5966
     18        0.2888        0.3026  2.6069
     19        0.2822        0.2960  2.5649
     20        0.2759        0.2903  2.6416


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        2.2296        2.1018  2.6030
      2        1.7747        1.3105  2.7862
      3        0.9944        0.7537  3.0341
      4        0.6626        0.5716  3.0153
      5        0.5383        0.4869  2.6265
      6        0.4754        0.4382  2.7852
      7        0.4351        0.4062  2.5867
      8        0.4083        0.3831  2.6174
      9        0.3882        0.3663  2.5803
     10        0.3705        0.3522  2.6381
     11        0.3578        0.3401  2.6284
     12        0.3448        0.3300  2.9316
     13        0.3341        0.3211  2.8183
     14        0.3260        0.3129  2.6816
     15        0.3164        0.3050  2.9119
     16        0.3088        0.2980  2.8344
     17        0.2998        0.2915  3.0824
     18        0.2945        0.2854  2.9193
     19        0.2880        0.2796  2.7288
     20        0.2801        0.2738  2.8736


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


...

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        2.3060        2.3012  1.4941
      2        2.2977        2.2935  1.4742
      3        2.2909        2.2870  1.4873
      4        2.2851        2.2815  1.4359
      5        2.2800        2.2766  1.5698
      6        2.2755        2.2721  1.5058
      7        2.2713        2.2679  1.4819
      8        2.2674        2.2639  1.5448
      9        2.2635        2.2600  1.5345
     10        2.2598        2.2562  1.6712
     11        2.2563        2.2526  1.6734
     12        2.2528        2.2491  1.6082
     13        2.2495        2.2457  1.6261
     14        2.2461        2.2422  1.6560
     15        2.2428        2.2388  1.6548
     16        2.2393        2.2353  1.6498
     17        2.2360        2.2317  1.7034
     18        2.2323        2.2280  1.5940
     19        2.2286        2.2241  1.6374
     20        2.2248        2.2201  1.6136


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.8059        0.3585  3.7507
      2        0.3072        0.2645  3.9199
      3        0.2318        0.2139  3.8832
      4        0.1859        0.1845  3.9805
      5        0.1552        0.1596  3.8423
      6        0.1341        0.1441  3.6881
      7        0.1185        0.1326  3.6692
      8        0.1065        0.1187  3.5928
      9        0.0961        0.1141  3.5038
     10        0.0883        0.1072  3.7005
     11        0.0819        0.1014  3.6699
     12        0.0757        0.0987  3.8550
     13        0.0719        0.0964  4.1255
     14        0.0674        0.0950  4.0064
     15        0.0639        0.0895  3.9249
     16        0.0611        0.0868  3.9312
     17        0.0582        0.0860  3.8800
     18        0.0561        0.0854  3.8113
     19        0.0533        0.0848  3.9613
     20        0.0513        0.0854  3.9774


GridSearchCV(cv=5,
             estimator=<class '__main__.RegularizedNet'>[initialized](
  module_=MyModule(
    (first): Linear(in_features=784, out_features=400, bias=True)
    (relu): ReLU()
    (second): Linear(in_features=400, out_features=150, bias=True)
    (last): Linear(in_features=150, out_features=10, bias=True)
    (Last): LogSoftmax(dim=None)
    (dropout): Dropout(p=0.1, inplace=False)
  ),
),
             param_grid={'lr': [0.01, 0.1, 0.001],
                         'module__layer1size': [400, 200],
                         'module__layer2size': [150, 70],
                         'optimizer__weight_decay': [0.001, 0.1]},
             scoring=<function my_scoring at 0x0000013B35DD9820>)

In [90]:

#Report Best Parameters
print(gs.best_score_, gs.best_params_)

0.9710666666666666 {'lr': 0.1, 'module__layer1size': 400, 'module__layer2size': 150, 'optimizer__weight_decay': 0.001}


# Problem 5.2

In [59]:
'''
Step - Now we are going to train the network with L1 regularization instead of L2 and dropout,
we are going to create a new network with a lambda1 parameter different than 0
- Keep the rest of the parameters you used in the previous network but dropout and L2 parameters are 0 
'''

#module=MyModule, criterion=torch.nn.NLLLoss, 
 #                       optimizer=torch.optim.SGD, lr = 0.13, lambda1 = 0,  module__dropout = 0.25,
  #                      optimizer__weight_decay = 0, module__input_size = 784, module__layer1size = 400,
   #                     module__layer2size = 150, module__num_classes = 10

new_net_l1 = RegularizedNet(module=MyModule, criterion=torch.nn.NLLLoss, 
                        optimizer= torch.optim.SGD, lr = 0.1, lambda1 = 0.2,  module__dropout = 0,
                        optimizer__weight_decay = 0, module__layer1size = 400, module__layer2size = 150,
                        module__input_size = 784, module__num_classes = 10)

In [32]:
#Refer to https://skorch.readthedocs.io/en/stable/user/save_load.html

import pickle


In [60]:
#Transfer Learning - 
#The following code will transfer the weights from L2 trained networks to initialize the new network before L1 training

'''

Notes - I assumed you have trained your L2 network using Skorch's NeuralNetClassifier
        I assume your trained model object is called "new_net"

'''

#Step - 1 - Save weights from L2 network

new_net.save_params(f_params='some-file.pkl') #This comes after net.fit(). You are saving the model weights in a pickle
#new_net_l1.save_params(f_params='some-file.pkl') #This comes after net.fit(). You are saving the model weights in a pickle



In [61]:

#Step - 2

new_net_l1.initialize()
new_net_l1.load_params(f_params='some-file.pkl')

In [ ]:
'''
Step train the network with the weights transfered from new_net, and perform grid search for the lambda1 parameter
'''

In [75]:
grid1 = {
    'lambda1': [0.01,0.001,0.0001]
}

In [76]:
gs1 = GridSearchCV(new_net_l1, grid1, refit=True, cv=5, scoring=my_scoring)

In [77]:
gs1.fit(X_train, y_train)

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        7.2924        4.1876  2.0927
      2        4.1811        4.1874  2.1407
      3        4.1811        4.1875  2.1124
      4        4.1811        4.1875  2.0674
      5        4.1811        4.1874  2.2325
      6        4.1811        4.1874  2.1393
      7        4.1811        4.1875  2.1480
      8        4.1811        4.1874  2.2938
      9        4.1811        4.1875  2.2924
     10        4.1811        4.1875  2.3309


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        7.3001        4.1906  2.2757
      2        4.1811        4.1906  2.3443
      3        4.1811        4.1907  2.2794
      4        4.1811        4.1907  2.3630
      5        4.1811        4.1907  2.2978
      6        4.1811        4.1907  2.4357
      7        4.1811        4.1907  2.4016
      8        4.1811        4.1906  2.3335
      9        4.1811        4.1906  2.3540
     10        4.1811        4.1905  2.4262


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


...

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.5655        1.0791  2.8003
      2        0.9578        0.8885  2.7682
      3        0.8151        0.7628  3.1329
      4        0.7042        0.6661  2.9126
      5        0.6142        0.5877  2.8621
      6        0.5402        0.5228  2.9683
      7        0.4790        0.4695  2.9232
      8        0.4283        0.4257  2.8494
      9        0.3866        0.3908  2.7251
     10        0.3530        0.3634  2.6480


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.4667        0.9962  3.5622
      2        0.8967        0.8228  3.5364
      3        0.7435        0.7008  3.3850
      4        0.6259        0.6050  3.3843
      5        0.5332        0.5241  3.4136
      6        0.4598        0.4581  3.5271
      7        0.4022        0.4061  3.5933
      8        0.3576        0.3678  3.4271
      9        0.3243        0.3395  3.3720
     10        0.3005        0.3201  3.3579


GridSearchCV(cv=5,
             estimator=<class '__main__.RegularizedNet'>[initialized](
  module_=MyModule(
    (first): Linear(in_features=784, out_features=400, bias=True)
    (relu): ReLU()
    (second): Linear(in_features=400, out_features=150, bias=True)
    (last): Linear(in_features=150, out_features=10, bias=True)
    (Last): LogSoftmax(dim=None)
    (dropout): Dropout(p=0, inplace=False)
  ),
),
             param_grid={'lambda1': [0.01, 0.001, 0.0001]},
             scoring=<function my_scoring at 0x0000013B35DD9820>)

In [78]:
print(gs1.best_score_, gs1.best_params_)

0.9534833333333333 {'lambda1': 0.0001}


In [119]:
'''
Step train the network with default initialization parameters
we can simply initialize with the same code as before (make sure to use the same parameters)

perform grid search for the lambda1 parameter as in the previous cell
'''

#new_net_l1 = RegularizedNet(module=MyModule, criterion=torch.nn.NLLLoss, 
 #                       optimizer= ..., lr = 0.001, lambda1 = ...,  module__dropout = 0,
  #                      optimizer__weight_decay = 0)

new_net_L1 = RegularizedNet(module=MyModule, criterion=torch.nn.NLLLoss, 
                        optimizer= torch.optim.SGD, lr = 0.1, lambda1 = 0.0001,  module__dropout = 0,
                        optimizer__weight_decay = 0, module__layer1size = 400, module__layer2size = 150,
                        module__input_size = 784, module__num_classes = 10)





In [120]:
gsL = GridSearchCV(new_net_L1, grid1, refit=True, cv=5, scoring=my_scoring)

In [121]:
gsL.fit(X_train, y_train)

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        7.2935        4.1866  2.2945
      2        4.1811        4.1865  2.6494
      3        4.1811        4.1865  2.5838
      4        4.1811        4.1865  2.5092
      5        4.1811        4.1866  2.3934
      6        4.1811        4.1866  2.6351
      7        4.1811        4.1866  2.4452
      8        4.1811        4.1866  2.4056
      9        4.1811        4.1866  2.5166
     10        4.1811        4.1866  2.2657


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        7.2970        4.1864  2.7059
      2        4.1811        4.1864  2.5966
      3        4.1811        4.1865  2.2601
      4        4.1811        4.1865  2.2234
      5        4.1811        4.1865  2.3745
      6        4.1811        4.1866  2.6532
      7        4.1811        4.1866  2.3174
      8        4.1811        4.1867  2.6290
      9        4.1811        4.1867  2.6722
     10        4.1811        4.1868  2.7579


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


...

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.5701        1.0814  2.6142
      2        0.9558        0.8865  2.5775
      3        0.8123        0.7611  2.5391
      4        0.7019        0.6643  2.6732
      5        0.6127        0.5860  2.6024
      6        0.5395        0.5222  2.8772
      7        0.4792        0.4703  2.9598
      8        0.4293        0.4275  2.9538
      9        0.3884        0.3933  2.9020
     10        0.3553        0.3666  2.8105


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.4545        0.9931  3.5223
      2        0.8930        0.8181  3.4919
      3        0.7397        0.6998  3.5059
      4        0.6228        0.6049  3.4984
      5        0.5306        0.5238  3.4575
      6        0.4577        0.4584  3.8277
      7        0.4007        0.4065  3.3553
      8        0.3569        0.3692  3.4567
      9        0.3243        0.3429  3.4439
     10        0.3010        0.3240  3.3019


GridSearchCV(cv=5,
             estimator=<class '__main__.RegularizedNet'>[uninitialized](
  module=<class '__main__.MyModule'>,
  module__dropout=0,
  module__input_size=784,
  module__layer1size=400,
  module__layer2size=150,
  module__num_classes=10,
),
             param_grid={'lambda1': [0.01, 0.001, 0.0001]},
             scoring=<function my_scoring at 0x0000013B35DD9820>)

In [122]:
print(gsL.best_score_, gsL.best_params_)

0.9520833333333334 {'lambda1': 0.0001}


# Problem 5.3

In [ ]:
'''
Keeping all the parameters for which you have got the best results before
try different optimizers.

Basically create the same new_net_l1 or new_net but train it with the requested optimizers in the pdf

GridSeacrh is not required but you can do it if you want for the different parameters of the optimizers

Notice you already train it with SGD in the previous problems
'''



In [ ]:
#module=MyModule, criterion=torch.nn.NLLLoss, 
 #                       optimizer=torch.optim.SGD, lr = 0.13, lambda1 = 0,  module__dropout = 0.25,
  #                      optimizer__weight_decay = 0, module__input_size = 784, module__layer1size = 400,
   #                     module__layer2size = 150, module__num_classes = 10

In [95]:
# Adam (look how I defined optimizer)

# It seems like we got the better results using L2.

New_net = RegularizedNet(module=MyModule, criterion=torch.nn.NLLLoss, 
                        optimizer= torch.optim.Adam, lr = 0.1, lambda1 = 0,  module__dropout = 0,
                        optimizer__weight_decay = 0, module__input_size = 784, module__layer1size = 400,
                        module__layer2size = 150, module__num_classes = 10)

'''
-Step now fit it and print the accuracy as in problem 1
'''

'\n-Step now fit it and print the accuracy as in problem 1\n'

In [96]:
New_net.fit(X_train, y_train, epochs=20)

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        2.3317        1.2852  3.2810
      2        1.3450        1.2928  3.5416
      3        1.2901        1.3178  5.0029
      4        1.4899        1.2855  4.7790
      5        1.4188        1.6250  4.7682
      6        1.4615        1.4607  4.9151
      7        1.3927        1.4401  5.0969
      8        1.3339        1.2875  4.8465
      9        1.2941        1.2607  4.9502
     10        1.3221        1.7542  4.9386
     11        1.5108        1.6079  5.0767
     12        1.6297        1.6164  4.9303
     13        1.5980        1.5914  5.4667
     14        1.6852        1.6481  5.3050
     15        1.6295        1.6172  5.1443
     16        1.6912        1.6748  4.9787
     17        1.6617        1.8374  4.9209
     18        1.6189        1.5713  5.4282
     19        1.6590        1.6127  5.3419
     20        1.7034        1.7079  5.1151


<class '__main__.RegularizedNet'>[initialized](
  module_=MyModule(
    (first): Linear(in_features=784, out_features=400, bias=True)
    (relu): ReLU()
    (second): Linear(in_features=400, out_features=150, bias=True)
    (last): Linear(in_features=150, out_features=10, bias=True)
    (Last): LogSoftmax(dim=None)
    (dropout): Dropout(p=0, inplace=False)
  ),
)

In [98]:
ya_pred_probs = New_net.predict(X_test)
ya_pred = torch.tensor([np.argmax(ya_pred_probs[i]) for i in range(ya_pred_probs.shape[0])])
ya_pred = ya_pred.type(torch.LongTensor)

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


In [99]:
accuracy_score(y_test,ya_pred)

0.2557

In [ ]:
# SGD with momentum 

In [100]:
New_net2 = RegularizedNet(module=MyModule, criterion=torch.nn.NLLLoss, 
                        optimizer= torch.optim.SGD, lr = 0.1, lambda1 = 0,  module__dropout = 0,
                        optimizer__weight_decay = 0, module__input_size = 784, module__layer1size = 400,
                        module__layer2size = 150, module__num_classes = 10, optimizer__momentum = 0.9)

In [101]:
New_net2.fit(X_train, y_train, epochs=20)

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.3799        0.1908  3.0065
      2        0.1303        0.1152  3.1291
      3        0.0874        0.1540  3.1424
      4        0.0623        0.2133  3.1813
      5        0.0445        0.1166  3.1655
      6        0.0304        0.0873  3.0386
      7        0.0213        0.1132  3.1019
      8        0.0154        0.0944  3.1153
      9        0.0165        0.0893  3.1658
     10        0.0092        0.0994  3.2538
     11        0.0108        0.1038  3.1913
     12        0.0119        0.1074  3.3298
     13        0.0089        0.1728  3.5283
     14        0.0107        0.1106  3.3284
     15        0.0093        0.1225  3.1139
     16        0.0088        0.1273  3.4461
     17        0.0048        0.1159  3.4036
     18        0.0080        0.1605  3.3544
     19        0.0089        0.1139  3.4482
     20        0.0025        0.1175  3.3114


<class '__main__.RegularizedNet'>[initialized](
  module_=MyModule(
    (first): Linear(in_features=784, out_features=400, bias=True)
    (relu): ReLU()
    (second): Linear(in_features=400, out_features=150, bias=True)
    (last): Linear(in_features=150, out_features=10, bias=True)
    (Last): LogSoftmax(dim=None)
    (dropout): Dropout(p=0, inplace=False)
  ),
)

In [102]:
yb_pred_probs = New_net2.predict(X_test)
yb_pred = torch.tensor([np.argmax(yb_pred_probs[i]) for i in range(yb_pred_probs.shape[0])])
yb_pred = yb_pred.type(torch.LongTensor)

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


In [103]:
accuracy_score(y_test,yb_pred)

0.9802

In [ ]:
# AdaGrad

In [113]:
New_net3 = RegularizedNet(module=MyModule, criterion=torch.nn.NLLLoss, 
                        optimizer= torch.optim.Adagrad, lr = 0.1, lambda1 = 0,  module__dropout = 0,
                        optimizer__weight_decay = 0, module__input_size = 784, module__layer1size = 400,
                        module__layer2size = 150, module__num_classes = 10)

In [116]:
New_net3.fit(X_train, y_train)

Re-initializing module because the following parameters were re-set: dropout, input_size, layer1size, layer2size, num_classes.
Re-initializing optimizer because the following parameters were re-set: weight_decay.


<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.2395        0.2015  3.2542
      2        0.1334        0.1511  3.2636
      3        0.0916        0.1362  3.2229
      4        0.0684        0.1292  3.2316
      5        0.0527        0.1312  3.2452
      6        0.0409        0.1319  3.1513
      7        0.0322        0.1325  3.1861
      8        0.0252        0.1357  3.2486
      9        0.0199        0.1404  3.2110
     10        0.0157        0.1443  3.2743


<class '__main__.RegularizedNet'>[initialized](
  module_=MyModule(
    (first): Linear(in_features=784, out_features=400, bias=True)
    (relu): ReLU()
    (second): Linear(in_features=400, out_features=150, bias=True)
    (last): Linear(in_features=150, out_features=10, bias=True)
    (Last): LogSoftmax(dim=None)
    (dropout): Dropout(p=0, inplace=False)
  ),
)

In [117]:
yc_pred_probs = New_net3.predict(X_test)
yc_pred = torch.tensor([np.argmax(yc_pred_probs[i]) for i in range(yc_pred_probs.shape[0])])
yc_pred = yc_pred.type(torch.LongTensor)

<ipython-input-9-bb6654121bf9>:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.Last(out)


In [118]:
accuracy_score(y_test,yc_pred)

0.9686